In [ ]:
import os
os.chdir("/home1/wangtianshu/universal-blocker")
from pathlib import Path

DATA_DIR = Path("./data/gittables/")
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

from joblib import Memory, Parallel, delayed
memory = Memory("../.cache")

In [ ]:
import pickle
import json
from collections import Counter, defaultdict

from pyarrow import parquet
from tqdm.notebook import tqdm

@memory.cache
def get_parquet_files(p):
    files = list(p.rglob("*.parquet"))
    return files

@memory.cache
def get_column_type_counters():
    counter_dict = defaultdict(Counter)
    def cal_column_types(f):
        try:
            schema = parquet.read_schema(f)
        except Exception:
            pass

        metadata = schema.metadata
        gittables_metadata = json.loads(metadata[b'gittables'].decode("utf-8"))
        # print(json.dumps(gittables_metadata, indent=2))

        for ann in ["dbpedia_syntactic_column_types",
                "schema_syntactic_column_types",
                "dbpedia_semantic_column_types",
                "schema_semantic_column_types"]:
            for _, v in gittables_metadata[ann].items():
                counter_dict[ann][v["cleaned_label"]] += 1

    Parallel(n_jobs=16, require="sharedmem")(delayed(cal_column_types)(f) for f in tqdm(parquet_files))

    return counter_dict

parquet_files = get_parquet_files(RAW_DIR)
counter_dict = get_column_type_counters()

In [ ]:
import json
import random

import pandas as pd
from pyarrow import parquet
from tqdm.notebook import tqdm

def cal_column_types(f):
    try:
        schema = parquet.read_schema(f)
    except Exception:
        return
    
    metadata = schema.metadata
    gittables_metadata = json.loads(metadata[b'gittables'].decode("utf-8"))
#     print(json.dumps(gittables_metadata, indent=2))
    if len(gittables_metadata["dbpedia_syntactic_column_types"]) > 1:

#         print(gittables_metadata["dbpedia_syntactic_column_types"])
        if gittables_metadata["csv_url"].startswith("https://github.com/alixaxel/dump.HN"):
            return

        df = pd.read_parquet(f)
        
        if list(df.columns) == ['ID', 'Type', 'Story', 'Parent', 'Points', 'Comments', 'Author',
       'Title', 'URL', 'Content', 'Created']:
            print(gittables_metadata["csv_url"])

        print(f)
        print(gittables_metadata["csv_url"])
        print(df)
        print()
        
        #         return f

# print(len(list(filter(lambda x: x is not None, Parallel(n_jobs=16)(delayed(cal_column_types)(f) for f in parquet_files)))))

Parallel(n_jobs=1)(delayed(cal_column_types)(f) for f in random.sample(parquet_files, 10))
# Parallel(n_jobs=1)(delayed(cal_column_types)(f) for f in [Path("data/gittables/raw/id_tables_licensed/FM-405_20171211_00-23.parquet")])

# TODO:
# 1. remove no ascii/english tables data/gittables/raw/id_tables_licensed/%EA%B0%B8%EC%98%A4%20%EC%B9%B4%ED%98%B8%203%20%E7%99%BA%E8%A6%8B%EF%BC%81%E3%81%B5%E3%81%9F%E3%82%8A%E3%81%AE%E5%85%B1%E9%80%9A%E7%82%B9_1.parquet
# 2. remove all number tables data/gittables/raw/id_tables_licensed/gw_19758.parquet data/gittables/raw/terminal_velocity_tables_licensed/result_23.parquet
#    str columns <= number columns / 3 data/gittables/raw/id_tables_licensed/0022000842_1.parquet
# 3. 高度相似表格

In [ ]:
data_dir = "./data/gittables/processed/"
from datasets.load import load_dataset
from torch.utils.data import DataLoader

tables = load_dataset(
    "json",
    data_dir=data_dir,
    split="train",
    streaming=True,
)
tables = tables.with_format("torch")
dataloader = DataLoader(
    dataset=tables,
    batch_size=128,
    num_workers=0,
    pin_memory=True,
    shuffle=False,
)

prefix_dir = Path("./data/gittables/raw_4943312/")   
for i, batch in tqdm(enumerate(dataloader)):
    if i >= 2400 and i <= 2500:
        files = set(batch["_file"])
        for f in files:
            f = prefix_dir / f
            df = pd.read_parquet(f)
            print(df.iloc[0].to_json(orient="records")[1:-1], end="\n\n")

#         print(batch["_file"])
    if i > 2600:
        break

In [ ]:
import json
import random

import pandas as pd
from pyarrow import parquet
from tqdm.notebook import tqdm

def cal_column_types(f):
    try:
        schema = parquet.read_schema(f)
    except Exception:
        return
    
    metadata = schema.metadata
    gittables_metadata = json.loads(metadata[b'gittables'].decode("utf-8"))
#     print(json.dumps(gittables_metadata, indent=2))
    if len(gittables_metadata["dbpedia_syntactic_column_types"]) > 1:
        print(gittables_metadata["dbpedia_syntactic_column_types"])

#         print(gittables_metadata["dbpedia_syntactic_column_types"])
        if gittables_metadata["csv_url"].startswith("https://github.com/alixaxel/dump.HN"):
            return

        df = pd.read_parquet(f)
        
        if list(df.columns) == ['ID', 'Type', 'Story', 'Parent', 'Points', 'Comments', 'Author',
       'Title', 'URL', 'Content', 'Created']:
            print(gittables_metadata["csv_url"])

        print(f)
        print(gittables_metadata["csv_url"])
        print(df)
        print()
        
        #         return f

# print(len(list(filter(lambda x: x is not None, Parallel(n_jobs=16)(delayed(cal_column_types)(f) for f in parquet_files)))))

# Parallel(n_jobs=1)(delayed(cal_column_types)(f) for f in random.sample(parquet_files, 10))
Parallel(n_jobs=1)(delayed(cal_column_types)(f) for f in [Path("data/gittables/raw/id_tables_licensed/FM-405_20171211_00-23.parquet")])

# TODO:
# 1. remove no ascii/english tables data/gittables/raw/id_tables_licensed/%EA%B0%B8%EC%98%A4%20%EC%B9%B4%ED%98%B8%203%20%E7%99%BA%E8%A6%8B%EF%BC%81%E3%81%B5%E3%81%9F%E3%82%8A%E3%81%AE%E5%85%B1%E9%80%9A%E7%82%B9_1.parquet
# 2. remove all number tables data/gittables/raw/id_tables_licensed/gw_19758.parquet data/gittables/raw/terminal_velocity_tables_licensed/result_23.parquet
#    str columns <= number columns / 3 data/gittables/raw/id_tables_licensed/0022000842_1.parquet
# 3. 高度相似表格